# Идея проекта, выборка и предобработка данных
<hr>

## Введение

```Текущая accuracy: 60-70%```

Проект представляет из себя анализатор дипфейков на основе анализа артефактов в различных зонах лица

В ходе предварительного анализа предметной области было выделено 3 наиболее перспективных для реализации метода анализа дипфейков:

- анализ движений глаз (саккад)
- анализ синзронизации речи и движений губ
- анализ артефактов

Так как анализ движений возможен только в динамике и довольно сложен в реализации, было решено реализовать метод анализа артефактов на статических изображениях (фото)

- В качестве "источника вдохновения" использовался [проект](https://github.com/rakshitsakhuja/Detecting-Deepfakes-with-OpenCV/blob/master/1.%20Processing%20Videos%20and%20Face%20Detection.ipynb), доступный по ссылке: 

https://github.com/rakshitsakhuja/Detecting-Deepfakes-with-OpenCV/blob/master/1.%20Processing%20Videos%20and%20Face%20Detection.ipynb

- В качестве датасета использовались фрагменты датасета [Deep Fake Detection Challenge DFDC](https://www.kaggle.com/competitions/deepfake-detection-challenge):

https://www.kaggle.com/competitions/deepfake-detection-challenge

- Про [распознанвание зон лица](https://pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/) можно прочесть по ссылке:

https://pyimagesearch.com/2017/04/10/detect-eyes-nose-lips-jaw-dlib-opencv-python/

- Сжатый [датасет 300 Faces In-the-Wild Challenge (300-W)](https://ibug.doc.ic.ac.uk/resources/300-W/) для распознавания точек на лице и аннотации к нему:

https://ibug.doc.ic.ac.uk/resources/300-W/

## Пайплайн проекта

Проект работает в следующем порядке:

- datacollector.py, faceparts.py:
1. Считывание по 1 кадру из видео-датасета DFDC (5 x 10 GB, 5 x 1700 видео)
2. Распознавание 68 точек на лице с помощью OpenCV, DLib и датасета 300-W
3. Отсев фото, которые не удалось распознать (остается около 3500 фото), запись *размера* лица и бинарного значения яркости *пикселей* в 68 зонах вокруг ключевых точек в их черно-белой версии (размер зон - 8-17% от ширины лица, сейчас подбирается) в dataset (5 х 0.2-0.5 GB)
> псевдокод: ```image.to_greyscale().cut([zone_schape])```

- notebook.ipynb (до метки "Legacy"):
1. Добавление гистограмм распределения яркости к склейке датасетов
2. Добавление метрик std и noise для каждой зоны
3. Очистка от исходных массивов пикселей, запись на диск (30 MB)

- notebook2.ipynb:
1. Отсекание лишних фейков из исходного датасета (остается 1000=500*2 из 3500 фото) для избавления от перекоса (по крайней мере в процессе подбора методов, такой датасет удобнее, иначе недо/переобучение вощникает слишком часто)
2. Сбор std и avg значений для каждого из классов => определение точек, с наибольшим значением ```|avg_fake-avg_real| / (std_fake + std_real)```
> sklearn.feature_selection.SelectKBest работает плохо, т.к. не включает метрики для всего лица, которые нужны для нормализации значений по точкам

3. Выборка самых показательных features + features по всему лицу
4. Обучение моделей

> Так как 5 из 8 моделей дают сопоставимую точность, на данном этапе целесообразно работать надо подбором фичей и размерами сканируемых зон, мб фильтрацией плохих/маленьких фото, тюнинг моделей (тщательный) следует проводть позднее

## Потенциальные темы НИРов

> 1 тема != 1 НИР, темы могут быть объединены или разбиты

- процесс сборки данных: введение про проблему и методы анализа дипфейков, датасет DFDC, распознавание точек, OpenCV, DLib, датасет 300-W
- Предобработка данных: фильтрация, гистограммы и агрегационные метрики
- Подбор features, метрики метрик, метрики по всему лицу
- Выбор и тюнинг моделей, влияние размера и перекоса датасета и т.д.

На более поздних этапах (опционально):
- выстраивание вокруг модели эконсистемы: веб-приложение, комплект клиентов на React-native / чат-бот, Google OAuth, Metamask Auth
- Биллинг для приложений, ценообразование, токены (можно крипту из тестнета (Goerli, Ether testnet) ради забавы прикрутить), мониторинг (Grafana?), Docker-Kuber, балансировка и прочие умные штуки

# Обработка данных
<hr>

> 990 фото, баланс классов 1:1, размер зон - 8..17% от ширины лица (в процессе экспериментов), f1-метрика на разных моделях - 0.60..0.70

## Импорт dataset-а и агрегация данных для классов 'fake' и 'real'

In [1]:
import os
os.getcwd()

'C:\\Users\\sergey.astakhov\\Desktop\\BmstuDeepFake'

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_json("../deep_fake_src/dfdc_dataframes/df_total_0_4_compact_frame_10.json")
# df = pd.read_json("../deep_fake_src/dfdc_dataframes/dataframe_compact_total_27_frame_10_lim100_offset0.json")

In [4]:
# Балансировка классов 1:1
# Инверсные метрики для полиномов

# df = df.drop(columns=['index','face_size_px'])
df = pd.concat([
    df[df.fake==True].sample(
        int(df[df.fake==False].shape[0]*1)), 
    df[df.fake==False]
])

face = df.filter(like='face', axis=1).copy()

for col in face:
    name = str(col) + '_reversed'
    df[name] = face[str(col)].map(lambda x: 1.0 / max(1,x))

df = df.filter(regex='^(.(?!(var)))*$', axis=1).filter(regex='^(.(?!(noise_1)))*$', axis=1)

print(df.shape)
# df = df.reset_index()
df.head()

(990, 281)


,index,filename,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
2949,330,mrctprnqnq.mp4,True,126736,18.212176,9.489084,14.302048,16.284267,16.674089,17.032550,...,1.171875,1.953125,16.406250,3.515625,0.124669,0.000008,0.025795,0.001127,1.0,1.0
2750,131,sujgbwtdks.mp4,True,24964,7.226494,9.733327,6.233311,8.694223,8.855275,13.490738,...,27.777778,58.333333,58.333333,44.444444,0.524756,0.000040,0.044072,0.005844,1.0,1.0
397,410,uqqivuldlu.mp4,True,198916,11.352545,19.382837,11.268391,10.407096,8.214142,12.333100,...,25.619835,19.008264,25.206612,20.247934,0.121659,0.000005,0.022213,0.001331,1.0,1.0
2811,192,flkmdghpzy.mp4,True,133956,27.280906,27.065286,28.254943,30.436147,30.415802,25.025741,...,21.296296,20.679012,19.444444,19.753086,0.117949,0.000007,0.025473,0.001305,1.0,1.0
2038,30,xxxrnkdsso.mp4,True,153664,10.684411,15.428025,7.590070,11.296266,18.007260,24.212321,...,15.740741,14.814815,9.567901,16.975309,0.161391,0.000007,0.018065,0.001597,1.0,1.0


In [5]:
# агрегация для фейков

df_fakes_compact = df[df.fake==True].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_fakes_reduced = pd.DataFrame(df_fakes_compact.mean()).T
df_fakes_reduced['fake'] = True
df_fakes_reduced = df_fakes_reduced.set_index('fake')
df_fakes_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
True,104235.822222,46.270812,549.235134,0.17565,0.286481,0.000015,0.023943,0.003179,0.996864,0.994378


In [6]:
# агрегация для реальных фото

df_real_compact = df[df.fake==False].filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ake)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
                                    
df_real_reduced = pd.DataFrame(df_real_compact.mean()).T
df_real_reduced['fake'] = False
df_real_reduced = df_real_reduced.set_index('fake')
df_real_reduced.filter(like='face', axis=1)

,face_size_px,overall_face_std,overall_face_hist_std,overall_face_hist_noise_0.5,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
fake,,,,,,,,,,
False,92126.343434,49.182392,534.771606,0.200743,0.308146,0.000016,0.022526,0.003057,0.999456,0.998436


In [7]:
# самые ярко-различающиеся по классам features

df_compare = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_compare['diff_rel'] = abs(df_compare[False] - df_compare[True]) / (df_compare[False] + df_compare[True])
df_compare['diff'] = abs(df_compare[False] - df_compare[True]) 

df_compare = df_compare.sort_values(by=['diff_rel'], ascending=False)
df_compare.head(15)

fake,False,True,diff_rel,diff
pt_57_hist_noise_0.5,0.871193,0.582470,0.198618,0.288723
pt_56_hist_noise_0.5,0.956043,0.656984,0.185402,0.299059
pt_21_hist_noise_0.5,1.300257,0.932893,0.164505,0.367364
index,541.650505,392.741414,0.159365,148.909091
pt_19_hist_noise_0.5,1.641023,1.197251,0.156353,0.443772
pt_11_hist_noise_0.5,1.231487,1.687239,0.156148,0.455752
pt_30_hist_noise_0.5,1.692450,1.247031,0.151530,0.445420
pt_57_hist_noise_3,16.263845,12.201226,0.142723,4.062620
pt_20_hist_noise_0.5,1.829038,1.384568,0.138309,0.444470
pt_67_hist_noise_0.5,1.935988,1.472110,0.136111,0.463878


In [8]:
# самые стабильные features

df_fakes_std = pd.DataFrame(df_fakes_compact.std()).T
df_fakes_std['fake'] = True
df_fakes_std = df_fakes_std.set_index('fake')

df_real_std = pd.DataFrame(df_real_compact.std()).T
df_real_std['fake'] = False
df_real_std = df_real_std.set_index('fake')

df_std = pd.concat([df_real_reduced, df_fakes_reduced]).T
df_std['total_std'] = (df_std[False] + df_std[True])

df_std = df_std.sort_values(by=['total_std'], ascending=True)
df_std.head(10)

fake,False,True,total_std
face_size_px_reversed,0.000016,0.000015,0.000031
overall_face_hist_std_reversed,0.003057,0.003179,0.006237
overall_face_std_reversed,0.022526,0.023943,0.046470
overall_face_hist_noise_0.5,0.200743,0.175650,0.376393
overall_face_hist_noise_3,0.308146,0.286481,0.594627
pt_8_hist_noise_0.5,0.507715,0.471264,0.978979
pt_7_hist_noise_0.5,0.540935,0.512301,1.053236
pt_6_hist_noise_0.5,0.688294,0.730736,1.419030
pt_9_hist_noise_0.5,0.687613,0.743008,1.430621
pt_57_hist_noise_0.5,0.871193,0.582470,1.453663


In [9]:
# самые надежные (разилчие_по_классам / разброс) features
# (var не кажется действительно надежной, скорее всего просто зависит от std квадратично или как-то еще)

df_smart = pd.concat([pd.DataFrame(df_compare['diff']),pd.DataFrame(df_std['total_std'])], axis=1)
df_smart['metric'] = df_smart['diff'] / df_smart['total_std']
df_smart = df_smart.sort_values(by=['metric'], ascending=False)
df_smart.head(30)

,diff,total_std,metric
pt_57_hist_noise_0.5,0.288723,1.453663,0.198618
pt_56_hist_noise_0.5,0.299059,1.613026,0.185402
pt_21_hist_noise_0.5,0.367364,2.233150,0.164505
index,148.909091,934.391919,0.159365
pt_19_hist_noise_0.5,0.443772,2.838275,0.156353
pt_11_hist_noise_0.5,0.455752,2.918725,0.156148
pt_30_hist_noise_0.5,0.445420,2.939481,0.151530
pt_57_hist_noise_3,4.062620,28.465071,0.142723
pt_20_hist_noise_0.5,0.444470,3.213606,0.138309
pt_67_hist_noise_0.5,0.463878,3.408099,0.136111


## Подбор features [to be modified]

> опирается на интуитивные предположения + метод определения фейков по шуму

In [10]:
features = list(set(
    list(df_smart.filter(like='face_hist', axis=0).index) +  # for relative metrics
    list(df_smart.filter(like='noise_0.5', axis=0).head(7).index) +
    list(df_smart.filter(like='noise_3', axis=0).head(7).index) +
    list(df_smart.filter(like='std', axis=0).head(5).index) +
    list(df_smart.filter(like='reversed', axis=0).index) 
    # list(df_smart.filter(like='var', axis=0).head(3).index) 
))

print(len(features))
features

27


['pt_57_hist_noise_0.5',
 'overall_face_hist_noise_3',
 'overall_face_std_reversed',
 'pt_37_hist_std',
 'pt_43_hist_noise_3',
 'pt_57_hist_noise_3',
 'pt_2_hist_std',
 'overall_face_hist_noise_0.5_reversed',
 'pt_11_hist_noise_0.5',
 'pt_19_hist_noise_0.5',
 'overall_face_hist_std',
 'overall_face_hist_std_reversed',
 'overall_face_hist_noise_3_reversed',
 'pt_3_hist_noise_3',
 'overall_face_hist_noise_0.5',
 'pt_20_hist_noise_0.5',
 'pt_58_hist_std',
 'pt_30_hist_noise_0.5',
 'pt_47_hist_noise_3',
 'pt_56_hist_noise_0.5',
 'face_size_px_reversed',
 'pt_21_hist_noise_0.5',
 'pt_57_hist_std',
 'pt_3_std',
 'pt_39_hist_noise_3',
 'pt_56_hist_noise_3',
 'pt_58_hist_noise_3']

## Разбивка данных на train и test сплиты

In [11]:
# очистка dataset-а от лишних колонок перед тренировкой модели (необязательно?)

df_compact = df.filter(regex='^(.(?!(raw)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ilename)))*$', axis=1) \
                                    .filter(regex='^(.(?!(ist_simple)))*$', axis=1)
df_compact.head(5)

,index,fake,face_size_px,pt_48_std,pt_49_std,pt_50_std,pt_51_std,pt_52_std,pt_53_std,pt_54_std,...,pt_13_hist_noise_3,pt_14_hist_noise_3,pt_15_hist_noise_3,pt_16_hist_noise_3,overall_face_hist_noise_3,face_size_px_reversed,overall_face_std_reversed,overall_face_hist_std_reversed,overall_face_hist_noise_0.5_reversed,overall_face_hist_noise_3_reversed
2949,330,True,126736,18.212176,9.489084,14.302048,16.284267,16.674089,17.032550,8.875852,...,1.171875,1.953125,16.406250,3.515625,0.124669,0.000008,0.025795,0.001127,1.0,1.0
2750,131,True,24964,7.226494,9.733327,6.233311,8.694223,8.855275,13.490738,6.745826,...,27.777778,58.333333,58.333333,44.444444,0.524756,0.000040,0.044072,0.005844,1.0,1.0
397,410,True,198916,11.352545,19.382837,11.268391,10.407096,8.214142,12.333100,11.254492,...,25.619835,19.008264,25.206612,20.247934,0.121659,0.000005,0.022213,0.001331,1.0,1.0
2811,192,True,133956,27.280906,27.065286,28.254943,30.436147,30.415802,25.025741,25.526887,...,21.296296,20.679012,19.444444,19.753086,0.117949,0.000007,0.025473,0.001305,1.0,1.0
2038,30,True,153664,10.684411,15.428025,7.590070,11.296266,18.007260,24.212321,14.877169,...,15.740741,14.814815,9.567901,16.975309,0.161391,0.000007,0.018065,0.001597,1.0,1.0


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [13]:
y = df_compact['fake']
X = pd.DataFrame(df_compact.drop(columns=['fake']))
# selector = SelectKBest(f_classif, k=7)
# X = selector.fit_transform(X, y)
X =  df_compact[features]

# mask = selector.get_support()
# X = X[X.columns[mask]]

# df_compact.filter(like='size', axis=0).head()

# print(df_compact[X.columns[mask]].shape)
# df_compact[X.columns[mask]].head()

pd.DataFrame(X).head()

,pt_57_hist_noise_0.5,overall_face_hist_noise_3,overall_face_std_reversed,pt_37_hist_std,pt_43_hist_noise_3,pt_57_hist_noise_3,pt_2_hist_std,overall_face_hist_noise_0.5_reversed,pt_11_hist_noise_0.5,pt_19_hist_noise_0.5,...,pt_30_hist_noise_0.5,pt_47_hist_noise_3,pt_56_hist_noise_0.5,face_size_px_reversed,pt_21_hist_noise_0.5,pt_57_hist_std,pt_3_std,pt_39_hist_noise_3,pt_56_hist_noise_3,pt_58_hist_noise_3
2949,0.00000,0.124669,0.025795,5.381537,8.593750,1.171875,7.280646,1.0,0.390625,0.781250,...,1.562500,8.203125,0.390625,0.000008,1.953125,5.051299,4.171963,14.062500,2.343750,4.296875
2750,0.00000,0.524756,0.044072,0.772924,19.444444,33.333333,0.670056,1.0,0.000000,0.000000,...,0.000000,27.777778,0.000000,0.000040,0.000000,0.589417,7.542882,27.777778,25.000000,16.666667
397,0.00000,0.121659,0.022213,4.884981,5.578512,0.619835,5.213016,1.0,2.479339,2.685950,...,1.652893,5.785124,0.206612,0.000005,0.413223,9.785779,29.589102,6.198347,0.826446,1.652893
2811,4.62963,0.117949,0.025473,2.251681,19.753086,11.419753,2.443053,1.0,5.864198,4.320988,...,2.469136,29.629630,6.481481,0.000007,2.469136,4.277456,11.824568,24.382716,13.271605,14.506173
2038,0.00000,0.161391,0.018065,3.585963,17.901235,1.234568,3.599011,1.0,2.777778,3.086420,...,1.851852,27.160494,0.617284,0.000007,0.925926,6.627929,8.123954,22.222222,1.851852,0.617284


In [14]:
# print(df_compact.filter(like='size', axis=1).shape)
# df_compact.filter(like='size', axis=1).head()

In [15]:
# sizes_col = X.filter(like='size', axis=1).copy()
# X = pd.concat([
#     df_compact[X.columns[mask]],
#     sizes_col
# ], axis=0)
# # X.join(df_compact.filter(like='size', axis=1), lsuffix='_caller', rsuffix='_other')
# X.head()

In [16]:
X_train, X_test, y_train, y_test = train_test_split( 
    X, 
    y,
    test_size=0.20, 
    random_state=420)

In [17]:
pd.DataFrame(X_train).head()

,pt_57_hist_noise_0.5,overall_face_hist_noise_3,overall_face_std_reversed,pt_37_hist_std,pt_43_hist_noise_3,pt_57_hist_noise_3,pt_2_hist_std,overall_face_hist_noise_0.5_reversed,pt_11_hist_noise_0.5,pt_19_hist_noise_0.5,...,pt_30_hist_noise_0.5,pt_47_hist_noise_3,pt_56_hist_noise_0.5,face_size_px_reversed,pt_21_hist_noise_0.5,pt_57_hist_std,pt_3_std,pt_39_hist_noise_3,pt_56_hist_noise_3,pt_58_hist_noise_3
1874,1.000000,0.100873,0.017269,3.100529,16.000000,3.000000,3.904625,1.0,4.750000,3.250000,...,0.500000,15.250000,0.250000,0.000005,1.750000,5.630900,41.480342,13.000000,3.000000,3.750000
2450,1.562500,0.163495,0.016640,2.137829,12.500000,15.234375,3.549868,1.0,0.781250,7.421875,...,5.859375,13.671875,1.953125,0.000009,2.734375,2.368412,18.097863,25.000000,12.890625,17.578125
1761,1.562500,0.098263,0.035070,4.047569,5.859375,3.906250,5.332682,1.0,5.468750,0.000000,...,3.515625,12.500000,0.781250,0.000008,0.781250,4.163645,3.724390,8.984375,3.125000,1.953125
572,0.147929,0.068918,0.024556,8.748423,7.100592,1.627219,10.605669,1.0,0.887574,1.035503,...,4.437870,6.804734,0.591716,0.000003,1.183432,8.818247,3.400133,5.473373,3.698225,1.627219
2329,2.734375,0.223081,0.014984,2.123161,12.109375,13.281250,1.944544,1.0,6.250000,7.031250,...,5.078125,14.453125,3.515625,0.000009,1.171875,2.926175,51.515386,26.562500,12.890625,25.000000


In [18]:
y_train.head()

1874    False
2450     True
1761     True
572      True
2329    False
Name: fake, dtype: bool

## Обучение моделей

### Служебные функции

In [19]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier


In [20]:
import joblib

In [21]:
def save_model(model, X_train, y_train, X_test, y_test, p=2, name='my_model.pkl'):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    joblib.dump(clf, name)

In [22]:
def train_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    clf = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=p), 
        model
    )
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return (
        confusion_matrix(y_test, y_pred), 
        classification_report(
            y_test, 
            y_pred, 
            target_names=['class "real"', 'class "fakes"'], 
            zero_division=np.nan)
    )

In [23]:
def print_polynomial_pipeline(model, X_train, y_train, X_test, y_test, p=2):
    confusion_matrix, classification_report = train_polynomial_pipeline(
        model, 
        X_train, 
        y_train, 
        X_test, 
        y_test,
        p
    )
    print(confusion_matrix)
    print(classification_report)

### SGDClassifier (f1=0.60) [to be modified]

In [24]:
print_polynomial_pipeline(
    SGDClassifier(),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
save_model(
    SGDClassifier(),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="sgd.pkl"
) 

[[72 29]
 [46 51]]
               precision    recall  f1-score   support

 class "real"       0.61      0.71      0.66       101
class "fakes"       0.64      0.53      0.58        97

     accuracy                           0.62       198
    macro avg       0.62      0.62      0.62       198
 weighted avg       0.62      0.62      0.62       198



### SVC (f1=0.64) [to be modified]

In [25]:
print_polynomial_pipeline(
    SVC(gamma='auto'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
save_model(
    SVC(gamma='auto'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="svc.pkl"
) 

[[53 48]
 [22 75]]
               precision    recall  f1-score   support

 class "real"       0.71      0.52      0.60       101
class "fakes"       0.61      0.77      0.68        97

     accuracy                           0.65       198
    macro avg       0.66      0.65      0.64       198
 weighted avg       0.66      0.65      0.64       198



### LogisticRegression (f1=0.61) [to be modified]

In [26]:
print_polynomial_pipeline(
    # LogisticRegression(max_iter=15000)
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
save_model(
    # LogisticRegression(max_iter=15000)
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="logistic_regression.pkl"
) 

[[66 35]
 [39 58]]
               precision    recall  f1-score   support

 class "real"       0.63      0.65      0.64       101
class "fakes"       0.62      0.60      0.61        97

     accuracy                           0.63       198
    macro avg       0.63      0.63      0.63       198
 weighted avg       0.63      0.63      0.63       198



### NN - MLPClassifier (f1=0.66) [to be modified]

In [27]:
print_polynomial_pipeline(
    # MLPClassifier(max_iter=5000), # 0.57
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(34,), # 5, 7, 7, 3 - 0.60
        random_state=1, 
        alpha=0.001, 
        # activation='relu',
        tol=1e-6,
        max_fun=15000,
        max_iter=15000),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
)
save_model(
    # MLPClassifier(max_iter=5000), # 0.57
    MLPClassifier(
        solver='lbfgs', 
        hidden_layer_sizes=(34,), # 5, 7, 7, 3 - 0.60
        random_state=1, 
        alpha=0.001, 
        # activation='relu',
        tol=1e-6,
        max_fun=15000,
        max_iter=15000),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1,
    name="mlp.pkl"
) 

[[72 29]
 [46 51]]
               precision    recall  f1-score   support

 class "real"       0.61      0.71      0.66       101
class "fakes"       0.64      0.53      0.58        97

     accuracy                           0.62       198
    macro avg       0.62      0.62      0.62       198
 weighted avg       0.62      0.62      0.62       198



### RandomForestClassifier (good, f1=0.64) [to be modified] 

In [28]:
print_polynomial_pipeline(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
) 
save_model(
    RandomForestClassifier(max_depth=7, random_state=42),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="random_forest.pkl"
) 

[[59 42]
 [30 67]]
               precision    recall  f1-score   support

 class "real"       0.66      0.58      0.62       101
class "fakes"       0.61      0.69      0.65        97

     accuracy                           0.64       198
    macro avg       0.64      0.64      0.64       198
 weighted avg       0.64      0.64      0.64       198



### DecisionTreeClassifier (f1=0.59) [to be modified]

In [29]:
print_polynomial_pipeline(
    DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[58 43]
 [32 65]]
               precision    recall  f1-score   support

 class "real"       0.64      0.57      0.61       101
class "fakes"       0.60      0.67      0.63        97

     accuracy                           0.62       198
    macro avg       0.62      0.62      0.62       198
 weighted avg       0.62      0.62      0.62       198



### RadiusNeighborsClassifier (f1=0.46) [уязвим к выбросам]

In [30]:
print_polynomial_pipeline(
    RadiusNeighborsClassifier(
        radius=100, 
        weights='distance', 
        p=1, 
        outlier_label='most_frequent'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)

[[34 67]
 [21 76]]
               precision    recall  f1-score   support

 class "real"       0.62      0.34      0.44       101
class "fakes"       0.53      0.78      0.63        97

     accuracy                           0.56       198
    macro avg       0.57      0.56      0.53       198
 weighted avg       0.58      0.56      0.53       198



### KNeighborsClassifier (f1=0.69) [to be modified]

In [31]:
print_polynomial_pipeline(
    KNeighborsClassifier(n_neighbors=4, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1
)
save_model(
    KNeighborsClassifier(n_neighbors=4, weights='distance'),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=1,
    name="knn.pkl"
) 

[[64 37]
 [29 68]]
               precision    recall  f1-score   support

 class "real"       0.69      0.63      0.66       101
class "fakes"       0.65      0.70      0.67        97

     accuracy                           0.67       198
    macro avg       0.67      0.67      0.67       198
 weighted avg       0.67      0.67      0.67       198



In [53]:
from sklearn.linear_model import Perceptron

In [58]:
mx_iter=10000
print_polynomial_pipeline(
    Perceptron(fit_intercept=True, max_iter=mx_iter, tol=None, shuffle=True, n_jobs=-1),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2
)
save_model(
    Perceptron(fit_intercept=True, max_iter=mx_iter, tol=None, shuffle=True, n_jobs=-1),
    X_train, 
    y_train, 
    X_test, 
    y_test,
    p=2,
    name="perceptron.pkl"
) 

[[66 35]
 [41 56]]
               precision    recall  f1-score   support

 class "real"       0.62      0.65      0.63       101
class "fakes"       0.62      0.58      0.60        97

     accuracy                           0.62       198
    macro avg       0.62      0.62      0.62       198
 weighted avg       0.62      0.62      0.62       198



In [98]:
estms = [
    SGDClassifier(),
    SVC(gamma='auto'),
    LogisticRegression(
        max_iter=15000, 
        # penalty=None,
        class_weight='balanced',
        solver='liblinear',
        # tol=1e-6
    ),
    # MLPClassifier(
    #     solver='lbfgs', 
    #     hidden_layer_sizes=(34,), # 5, 7, 7, 3 - 0.60
    #     random_state=1, 
    #     alpha=0.001, 
    #     # activation='relu',
    #     tol=1e-6,
    #     max_fun=15000,
    #     max_iter=15000
    # ),
    # Perceptron(fit_intercept=True, max_iter=10000, tol=None, shuffle=True),
    RandomForestClassifier(max_depth=7, random_state=42),
    DecisionTreeClassifier(max_depth=5, min_samples_split=10, min_samples_leaf=5),
    RadiusNeighborsClassifier(
        radius=100, 
        weights='distance', 
        p=1, 
        outlier_label='most_frequent'
    ),
    KNeighborsClassifier(n_neighbors=4, weights='distance')
]

estms2 = []

for model in estms:
    estms2.append(
        (
            str(model),
            make_pipeline(
                StandardScaler(),
                PolynomialFeatures(degree=2), 
                model
            )
        )
    )

In [99]:
# print(estms[0])

In [100]:
from sklearn.ensemble import StackingClassifier

In [101]:
md = LogisticRegression(max_iter=15000)
# md = KNeighborsClassifier()

clf = StackingClassifier(
    estimators=estms2, 
    final_estimator = make_pipeline(
        StandardScaler(),
        PolynomialFeatures(degree=2), 
        md
    ),
    stack_method = 'predict',
    n_jobs=-1,
    verbose=1
)

In [102]:
from datetime import datetime

In [103]:
print(datetime.now().strftime("%H:%M:%S"))

11:04:59


In [104]:
clf.fit(X_train, y_train)

StackingClassifier(estimators=[('SGDClassifier()',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('polynomialfeatures',
                                                 PolynomialFeatures()),
                                                ('sgdclassifier',
                                                 SGDClassifier())])),
                               ("SVC(gamma='auto')",
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('polynomialfeatures',
                                                 PolynomialFeatures()),
                                                ('svc', SVC(gamma='auto'))])),
                               ("LogisticRegression(...
                                                 StandardScaler()),
                                                ('polynomialfeatures',
                                                 PolynomialFeatures()),
                                                ('kneighborsclassifier',
                                                 KNeighborsClassifier(n_neighbors=4,
                                                                      weights='distance'))]))],
                   final_estimator=Pipeline(steps=[('standardscaler',
                                                    StandardScaler()),
                                                   ('polynomialfeatures',
                                                    PolynomialFeatures()),
                                                   ('logisticregression',
                                                    LogisticRegression(max_iter=15000))]),
                   n_jobs=-1, stack_method='predict', verbose=1)

In [105]:
y_pred = clf.predict(X_test)
print(datetime.now().strftime("%H:%M:%S"))

11:05:16


In [106]:
confusion_matrix(y_test, y_pred)

array([[66, 35],
       [32, 65]], dtype=int64)

In [107]:
print(
    classification_report(
        y_test, 
        y_pred, 
        target_names=['class "real"', 'class "fakes"'], 
        zero_division=np.nan
    )
)

               precision    recall  f1-score   support

 class "real"       0.67      0.65      0.66       101
class "fakes"       0.65      0.67      0.66        97

     accuracy                           0.66       198
    macro avg       0.66      0.66      0.66       198
 weighted avg       0.66      0.66      0.66       198

